In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import libraries
import sys

sys.path.append("../src/bcc_model/")

import itertools
import math

import pandas as pd
from v1.blue_carbon_project import BlueCarbonProject
from v1.cost_calculator import CostCalculator
from v1.sequestration_credits_calculator import SequestrationCreditsCalculator
from v1.utils import generate_master_table

In [4]:
# Define functions:


def calculate_breakeven_cost(project, max_iterations=100, tolerance=1e-5):
    """
    Function to calculate the breakeven cost of carbon
    """
    # Start the calculation with the initial carbon price
    carbon_price = project.carbon_price

    for _iteration in range(max_iterations):
        # Update the project carbon price and initialize cost calculator
        project.carbon_price = carbon_price
        cost_calculator = CostCalculator(project)

        # Calculate NPV covering cost
        npv_covering_cost = cost_calculator.NPV_covering_cost
        credits_issued = cost_calculator.credits_issued

        # Check if the NPV covering cost is within the acceptable tolerance
        if abs(npv_covering_cost) < tolerance:
            # Retrieve additional cost estimates and summary
            cost_summary = cost_calculator.get_summary()  # Assuming this method exists
            cost_estimates = cost_calculator.get_cost_estimates()  # Assuming this method exists

            return {
                "breakeven_carbon_price": float(carbon_price),
                "cost_summary": cost_summary,
                "cost_estimates": cost_estimates,
            }

        # Ensure credits_issued is not zero to avoid division errors
        if credits_issued == 0:
            print("Error: Credits issued are zero, breakeven cost cannot be calculated.")
            return None

        # Update carbon price based on the NPV covering cost and credits issued
        carbon_price -= npv_covering_cost / credits_issued

    # If max_iterations are reached without convergence, return the last calculated price
    print("Warning: Max iterations reached without convergence.")
    return {
        "breakeven_carbon_price": float(carbon_price),
        "cost_summary": cost_calculator.get_summary(),
        "cost_estimates": cost_calculator.get_cost_estimates(),
    }

In [5]:
# Paths
master_data_path = "../excel/Carbon-Cost Data Upload.xlsm"
output_path = "../raw_data/output.csv"
raw_scorecard_data_path = "../raw_data/scorecard_rating/qualitative_scorecard_rating.csv"
output_scorecard = "../raw_data/output_scoring.csv"  # this needs to generate the ../excel/data_ingestion_project_scorecard.xlsm


In [6]:
# 1. Import master table, base size table and base increase table
#  with information for the project calculations
# data_path = "../raw_data/data_ingestion.xlsm"
# Import excel with the import datai
master_table = generate_master_table(master_data_path)
# Open the excel file - tab 'Base_size_table'
base_size = pd.read_excel(master_data_path, sheet_name="Base_size_table")

# Open the excel file - tab 'Base_increase'
base_increase = pd.read_excel(master_data_path, sheet_name="Base_increase")

# change in the country column "Caribbean" to "The Bahamas"
master_table["country"] = master_table["country"].replace("Caribbean", "The Bahamas")

master_table.head()

,country,country_code,activity,ecosystem,size_ha,feasibility_analysis_cost,planning_and_admin_cost,data_collection_and_field_cost,community_representation_liaison_cost,blue_carbon_project_planning_cost,...,extent,extent_historic,unprotected_extent,loss_rate,restorable_land,tier_1_ipcc_default_value,tier_2_country_specific_rate,tier_1_global_emission_factor,tier_2_country_specific_emission_factor_agb,tier_2_country_specific_emission_factor_soc
0,United States,USA,Restoration,Mangrove,500,100000,166766.666667,26666.666667,126500.000000,100000,...,42057.40171,231272.5243,20077.3163,-0.008868,9665.780512,5.94,17.2349,27.2,55.227609,58.002081
1,Indonesia,IDN,Restoration,Mangrove,500,50000,166766.666667,26666.666667,71183.333333,100000,...,2901690.307,2930508.086,2028220.962,-0.003716,204536.1679,5.94,29.5,27.2,355.398394,71.876642
2,Australia,AUS,Restoration,Mangrove,500,70000,166766.666667,26666.666667,113016.666667,115000,...,1004148.846,1013552.136,482831.9666,-0.001498,69160.39725,5.94,7.285,27.2,197.881108,59.762089
3,The Bahamas,BHS,Restoration,Mangrove,500,70000,166766.666667,26666.666667,103100.000000,125000,...,150839.0825,144788.5852,34035.8866,-0.007321,9842.169833,5.94,7.5,27.2,21.830982,107.946528
4,Kenya,KEN,Restoration,Mangrove,500,50000,166766.666667,26666.666667,66050.000000,100000,...,54121.50058,54080.01031,44742.4502,-0.002781,1609.6981,5.94,19,27.2,91.779817,79.040056


In [7]:
# 2. Define different list with all the possible values for the different parameters

# Definition of lists
country_list = [
    "United States",
    "Indonesia",
    "Australia",
    "The Bahamas",
    "Kenya",
    "Mexico",
    "Colombia",
    "India",
    "China",
]
# Include country size information to normalize the total cost
# Idially we will want to have this information in the country entity
country_size_ha = {
    "United States": 947084624.2706754,
    "Indonesia": 188785480.2259437,
    "Australia": 768882542.08165,
    "The Bahamas": 1338557.8163060332,
    "Kenya": 58606174.82755706,
    "Mexico": 195179334.58619106,
    "Colombia": 113742621.27637246,
    "India": 297769359.954299,
    "China": 934894938.3876103,
}
ecosystem_list = ["Mangrove", "Seagrass", "Salt marsh"]
activity_type_list = ["Restoration", "Conservation"]
restoration_activity_type_list = ["Planting", "Hydrology", "Hybrid"]

# Definition of parameters
project_size_filter = ["Small", "Medium", "Large"]
price_type = ["Opex breakeven", "Market price"]

# Define project size thresholds for each ecosystem and activity type
project_size_thresholds = {
    ("Mangrove", "Conservation"): {"Small": 4000, "Medium": 20000, "Large": 40000},
    ("Mangrove", "Restoration"): {"Small": 100, "Medium": 500, "Large": 1000},
    ("Salt marsh", "Conservation"): {"Small": 800, "Medium": 4000, "Large": 8000},
    ("Salt marsh", "Restoration"): {"Small": 100, "Medium": 500, "Large": 1000},
    ("Seagrass", "Conservation"): {"Small": 400, "Medium": 2000, "Large": 4000},
    ("Seagrass", "Restoration"): {"Small": 100, "Medium": 500, "Large": 1000},
}

In [8]:
# 3. Create all possible combinations for the overview page
all_combinations = []

for country, ecosystem, activity, project_size, price in itertools.product(
    country_list, ecosystem_list, activity_type_list, project_size_filter, price_type
):
    # Apply restoration activity type only if the activity is "Restoration"
    if activity == "Restoration":
        for restoration_activity in restoration_activity_type_list:
            threshold = project_size_thresholds.get((ecosystem, activity), {}).get(project_size)
            all_combinations.append(
                {
                    "country": country,
                    "ecosystem": ecosystem,
                    "activity": activity,
                    "activity_type": restoration_activity,
                    "project_size_filter": project_size,
                    "project_size": threshold,
                    "price_type": price,
                }
            )
    else:
        # For Conservation, restoration activity type should be None
        threshold = project_size_thresholds.get((ecosystem, activity), {}).get(project_size)
        all_combinations.append(
            {
                "country": country,
                "ecosystem": ecosystem,
                "activity": activity,
                "activity_type": None,
                "project_size_filter": project_size,
                "project_size": threshold,
                "price_type": price,
            }
        )

# Add country size information to the combinations
for combination in all_combinations:
    combination["country_size_ha"] = country_size_ha[combination["country"]]

# Convert to DataFrame
df_combinations = pd.DataFrame(all_combinations)

# Display the first few rows of the DataFrame
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type,country_size_ha
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08
1,United States,Mangrove,Restoration,Hydrology,Small,100,Opex breakeven,9.470846e+08
2,United States,Mangrove,Restoration,Hybrid,Small,100,Opex breakeven,9.470846e+08
3,United States,Mangrove,Restoration,Planting,Small,100,Market price,9.470846e+08
4,United States,Mangrove,Restoration,Hydrology,Small,100,Market price,9.470846e+08


In [9]:
# 4. Apply model constrains

# Remove all rows for Seagrass restoration projects that are not Planting
df_combinations = df_combinations[
    ~(
        (df_combinations["ecosystem"] == "Seagrass")
        & (df_combinations["activity"] == "Restoration")
        & (df_combinations["activity_type"] != "Planting")
    )
]

# Define conditions for removal based on country, ecosystem, activity, and activity_type
conditions_to_remove = [
    # List of country-ecosystem pairs to remove 'Hybrid' restoration activities
    # Remove the projects for Indonesia Manrgove Restoaration Hybrid
    # Remove projects for Indonesia Salt Marsh Restoration Hybrid
    # Remove projects for Australia Mangrove Restoration Hybrid
    # Remove pprojects for Australia Salt Marsh Restoration Hybrid
    # Remove projects for Caribbean Mangrove Restoration Hybrid
    # Remove project for Caribbean Salt marsh Restoration Hybrid
    # Remove projects for Kenya Mangrove Restoration Hybrid
    # Remove projects for Kenya Salt marsh Restoration Hybrid
    # Remove projects for Colombia mangrove restoration hybrid
    # Remove projects for Colombia Salt marsh Restoration Hybrid
    # Remove prjects India Manrgove restoration Hybrid
    # Remove projects for India Salt marsh Restoration Hybrid
    # Remove projects for China Mangrove Restoration Hybrid
    # Remove projects for China Salt marsh Restoration Hybrid
    ("Indonesia", "Mangrove"),
    ("Indonesia", "Salt marsh"),
    ("Australia", "Mangrove"),
    ("Australia", "Salt marsh"),
    ("Caribbean", "Mangrove"),
    ("Caribbean", "Salt marsh"),
    ("Kenya", "Mangrove"),
    ("Kenya", "Salt marsh"),
    ("Colombia", "Mangrove"),
    ("Colombia", "Salt marsh"),
    ("India", "Mangrove"),
    ("India", "Salt marsh"),
    ("China", "Mangrove"),
    ("China", "Salt marsh"),
]

# 3. Apply the conditions to filter out specified rows in a single operation
df_combinations = df_combinations[
    ~(
        (df_combinations["activity"] == "Restoration")
        & (df_combinations["activity_type"] == "Hybrid")
        & (
            df_combinations[["country", "ecosystem"]]
            .apply(tuple, axis=1)
            .isin(conditions_to_remove)
        )
    )
]

In [10]:
# 5. Get the country name, country_code and continent information from the master table:

# Filter master table just for the columns we need
master_table_filtered = master_table[["country", "country_code"]]
master_table_filtered.head()

# left merge the master_table with the df_combinations
df_combinations = df_combinations.merge(master_table_filtered, how="left", on=["country"])
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type,country_size_ha,country_code
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA
1,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA
2,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA
3,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA
4,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA


In [11]:
# 6. Get the project_size_ha from the master_table and rename it to base_size
base_size_value_df = master_table[["country_code", "ecosystem", "activity", "size_ha"]]

# Merge base_size_value_df with df_combinations
df_combinations = df_combinations.merge(
    base_size_value_df, how="left", on=["country_code", "ecosystem", "activity"]
)
# Rename the column project_size_ha to base_size
df_combinations.rename(columns={"size_ha": "base_size"}, inplace=True)
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type,country_size_ha,country_code,base_size
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500
1,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500
2,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500
3,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500
4,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500


In [12]:
# 7. Create a new column called "project_name" by concatenating the "country", "ecosystem",
# "activity" and "project_size" columns
df_combinations["project_name"] = (
    df_combinations["country"]
    + " "
    + df_combinations["ecosystem"]
    + " "
    + df_combinations["activity"]
    + " "
    + df_combinations["project_size"].astype(str)
)
# drop duplicates in case we have some duplicated project names
df_combinations.drop_duplicates(inplace=True)
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type,country_size_ha,country_code,base_size,project_name
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500,United States Mangrove Restoration 100
6,United States,Mangrove,Restoration,Hydrology,Small,100,Opex breakeven,9.470846e+08,USA,500,United States Mangrove Restoration 100
12,United States,Mangrove,Restoration,Hybrid,Small,100,Opex breakeven,9.470846e+08,USA,500,United States Mangrove Restoration 100
18,United States,Mangrove,Restoration,Planting,Small,100,Market price,9.470846e+08,USA,500,United States Mangrove Restoration 100
24,United States,Mangrove,Restoration,Hydrology,Small,100,Market price,9.470846e+08,USA,500,United States Mangrove Restoration 100


In [13]:
# 8. Run the model for each project and calculate the costs

# Step 1: Initialize columns for the output df with None
columns_to_add = [
    "total_cost_npv",
    "total_cost",
    "total_weighted_cost_npv",
    "total_weighted_cost",
    "capex_npv",
    "capex",
    "opex_npv",
    "opex",
    "country_abatement_potential",
    "project_abatement_potential",
    "cost_per_tco2e",
    "cost_per_tco2e_npv",
    "feasibility_analysis_npv",
    "feasibility_analysis",
    "conservation_planning_npv",
    "conservation_planning",
    "data_collection_npv",
    "data_collection",
    "community_representation_npv",
    "community_representation",
    "blue_carbon_project_planning_npv",
    "blue_carbon_project_planning",
    "establishing_carbon_rights_npv",
    "establishing_carbon_rights",
    "validation_npv",
    "validation",
    "implementation_labor_npv",
    "implementation_labor",
    "monitoring_maintenance_npv",
    "monitoring_maintenance",
    "community_benefit_npv",
    "community_benefit",
    "carbon_standard_fees_npv",
    "carbon_standard_fees",
    "baseline_reassessment_npv",
    "baseline_reassessment",
    "mrv_npv",
    "mrv",
    "long_term_project_operating_npv",
    "long_term_project_operating",
    "initial_price_assumption",
    "leftover_after_opex",
    "leftover_after_opex_npv",
    "total_revenue",
    "total_revenu_npv",
    "credits_issued",
]
df_combinations[columns_to_add] = None

for index, row in df_combinations.iterrows():
    projet_name = row["project_name"]
    print(f"Calculating project {projet_name}...")
    # initial_price_assumption = 30

    # Set up common BlueCarbonProject parameters
    project_params = {
        "activity": row["activity"],
        "ecosystem": row["ecosystem"],
        "country": row["country"],
        "master_table": master_table,
        "base_size": base_size,
        "base_increase": base_increase,
        # 'carbon_price': initial_price_assumption,
        "carbon_revenues_to_cover": "Opex",
        "project_size_ha": row["project_size"],
        "restoration_activity": row["activity_type"],
        "planting_success_rate": 0.8,
        "loss_rate_used": "National average",
    }

    # Set ecosystem-specific rates
    if row["ecosystem"] == "Mangrove":
        project_params.update(
            {
                "sequestration_rate_used": "Tier 2 - Country-specific rate",
                "emission_factor_used": "Tier 2 - Country-specific emission factor",
            }
        )
    else:
        print(f"No default Tier 2 sequestration rate data available for {row['ecosystem']}.")
        project_params.update(
            {
                "sequestration_rate_used": "Tier 1 - IPCC default value",
                "emission_factor_used": "Tier 1 - Global emission factor",
            }
        )

    # # Initialize project
    # project = BlueCarbonProject(**project_params)

    # Determine the initial carbon price based on price type
    initial_price_assumption = 30
    project_params["carbon_price"] = initial_price_assumption

    # Initialize project with the initial or market price
    project = BlueCarbonProject(**project_params)

    if row["price_type"] == "Market price":
        df_combinations.loc[index, "initial_price_assumption"] = initial_price_assumption
        # cost_calculator = CostCalculator(project)
        # summary = cost_calculator.get_summary()
        # cost_estimates = cost_calculator.get_cost_estimates()
    elif row["price_type"] == "Opex breakeven":
        try:
            breakeven = calculate_breakeven_cost(project, max_iterations=150)
            breakeven_cost = breakeven["breakeven_carbon_price"]
            # print(f"Breakeven cost for project {row['project_name']} is {breakeven_cost}")
            df_combinations.loc[index, "initial_price_assumption"] = breakeven_cost
            # summary = breakeven['cost_summary']
            # cost_estimates = breakeven['cost_estimates']
            # Update project_params with breakeven carbon price
            project_params["carbon_price"] = breakeven_cost

            # Re-initialize project with breakeven price
            project = BlueCarbonProject(**project_params)
        except:  # noqa: E722
            print(f"Error calculating breakeven cost for project {row['project_name']}")
            df_combinations.loc[index, "initial_price_assumption"] = None
            # Re-initialize project with the initial price if breakeven calculation fails
            project_params["carbon_price"] = (
                initial_price_assumption  # <-- Set to initial price if error
            )
            project = BlueCarbonProject(**project_params)
            # cost_calculator = CostCalculator(project)
            # summary = cost_calculator.get_summary()
            # cost_estimates = cost_calculator.get_cost_estimates()

    # print('project params:', project_params)
    # Step 3: Calculate costs and extract data into dictionary
    cost_calculator = CostCalculator(project)
    cost_estimates = cost_calculator.get_cost_estimates()
    summary = cost_calculator.get_summary()

    # Define a dictionary for cost components mapping
    cost_mapping = {
        "total_cost_npv": ("Total cost", "NPV"),
        "total_cost": ("Total cost", "Total cost"),
        "capex_npv": ("Capital expenditure", "NPV"),
        "capex": ("Capital expenditure", "Total cost"),
        "opex_npv": ("Operating expenditure", "NPV"),
        "opex": ("Operating expenditure", "Total cost"),
        "feasibility_analysis_npv": ("Feasibility analysis", "NPV"),
        "feasibility_analysis": ("Feasibility analysis", "Total cost"),
        "conservation_planning_npv": ("Conservation planning and admin", "NPV"),
        "conservation_planning": ("Conservation planning and admin", "Total cost"),
        "data_collection_npv": ("Data collection and field costs", "NPV"),
        "data_collection": ("Data collection and field costs", "Total cost"),
        "community_representation_npv": ("Community representation / liaison", "NPV"),
        "community_representation": ("Community representation / liaison", "Total cost"),
        "blue_carbon_project_planning_npv": ("Blue carbon project planning", "NPV"),
        "blue_carbon_project_planning": ("Blue carbon project planning", "Total cost"),
        "establishing_carbon_rights_npv": ("Establishing carbon rights", "NPV"),
        "establishing_carbon_rights": ("Establishing carbon rights", "Total cost"),
        "validation_npv": ("Validation", "NPV"),
        "validation": ("Validation", "Total cost"),
        "implementation_labor_npv": ("Implementation labor", "NPV"),
        "implementation_labor": ("Implementation labor", "Total cost"),
        "monitoring_npv": ("Monitoring", "NPV"),
        "maintenance_npv": ("Maintenance", "NPV"),
        "monitoring": ("Monitoring", "Total cost"),
        "maintenance": ("Maintenance", "Total cost"),
        "community_benefit_npv": ("Community benefit sharing fund", "NPV"),
        "community_benefit": ("Community benefit sharing fund", "Total cost"),
        "carbon_standard_fees_npv": ("Carbon standard fees", "NPV"),
        "carbon_standard_fees": ("Carbon standard fees", "Total cost"),
        "baseline_reassessment_npv": ("Baseline reassessment", "NPV"),
        "baseline_reassessment": ("Baseline reassessment", "Total cost"),
        "mrv_npv": ("MRV", "NPV"),
        "mrv": ("MRV", "Total cost"),
        "long_term_project_operating_npv": ("Long-term project operating", "NPV"),
        "long_term_project_operating": ("Long-term project operating", "Total cost"),
    }

    # Loop over each cost component and assign
    for col_name, (cost_label, column) in cost_mapping.items():
        df_combinations.loc[index, col_name] = (
            cost_estimates.loc[cost_estimates["Cost estimates (USD)"] == cost_label, column]
            .values[0]
            .replace("$", "")
            .replace(",", "")
        )

    # Add monitoring_maintenance_npv
    df_combinations.loc[index, "monitoring_maintenance_npv"] = float(
        df_combinations.loc[index, "monitoring_npv"]
    ) + float(df_combinations.loc[index, "maintenance_npv"])
    df_combinations.loc[index, "monitoring_maintenance"] = float(
        df_combinations.loc[index, "monitoring"]
    ) + float(df_combinations.loc[index, "maintenance"])

    # Abatement potential calculations
    sequestration_credits_calculator = SequestrationCreditsCalculator(project)
    # Calculate country abatement potential
    country_abatement_potential = round(
        sequestration_credits_calculator.calculate_abatement_potential()
    )
    df_combinations.loc[index, "country_abatement_potential"] = country_abatement_potential
    # Calculate project abatement potential
    net_emissions_reduction_plan = (
        sequestration_credits_calculator.calculate_net_emissions_reductions()
    )
    project_abatement_potential = round(sum(net_emissions_reduction_plan.values()))
    df_combinations.loc[index, "project_abatement_potential"] = project_abatement_potential

    # Weighted cost - divide total_cost and total_cost_npv by country size
    country_size = df_combinations.loc[index, "country_size_ha"]
    df_combinations.loc[index, "total_weighted_cost_npv"] = (
        float(df_combinations.loc[index, "total_cost_npv"]) / country_size
    )
    df_combinations.loc[index, "total_weighted_cost"] = (
        float(df_combinations.loc[index, "total_cost"]) / country_size
    )

    # Safely handle NaN in cost_per_tCO2e
    cost_per_tCO2e = cost_calculator.cost_per_tCO2e  # noqa: N816
    df_combinations.loc[index, "cost_per_tco2e"] = (
        round(cost_per_tCO2e) if not math.isnan(cost_per_tCO2e) else 0
    )

    # Safely handle NaN in cost_per_tCO2e NPV
    cost_per_tCO2e_npv = cost_calculator.cost_per_tCO2e  # noqa: N816
    df_combinations.loc[index, "cost_per_tco2e_npv"] = (
        round(cost_per_tCO2e) if not math.isnan(cost_per_tCO2e_npv) else 0
    )

    # Add total revenue, total revenue NPV, and credits issued
    df_combinations.loc[index, "total_revenue"] = (
        str(summary.get("Total revenue (non-discounted)", 0)).replace("$", "").replace(",", "")
    )
    df_combinations.loc[index, "total_revenu_npv"] = (
        str(summary.get("Total revenue (NPV)", 0)).replace("$", "").replace(",", "")
    )
    df_combinations.loc[index, "credits_issued"] = summary.get("Credits issued", 0)

    # # Add left_over_after_opex and left_over_after_opex_npv
    df_combinations["leftover_after_opex_npv"] = df_combinations["total_revenu_npv"].astype(
        float
    ) - df_combinations["opex_npv"].astype(float)
    df_combinations["leftover_after_opex"] = df_combinations["total_revenue"].astype(
        float
    ) - df_combinations["opex"].astype(float)

Calculating project United States Mangrove Restoration 100...


### Project Summary
| Parameter                            | Value                                             |
| ------------------------------------ | ------------------------------------------------- |
| Project | United States Mangrove Restoration (100 ha) |
| $/tCO2e (total cost, NPV) | $256 |
| $/ha | $56,489 |
| NPV covering cost | $-0 |
| IRR when priced to cover opex | 4.0% |
| IRR when priced to cover total costs | -11.7% |
| Total cost (NPV) | $5,648,912 |
| Capital expenditure (NPV) | $2,732,701 |
| Operating expenditure (NPV) | $2,916,212 |
| Credits issued | 22,061 |
| Total revenue (NPV) | $2,916,212 |
| Total revenue (non-discounted) | $4,921,553 |
| Financing cost | $149,513 |
| Funding gap (NPV) | $0 |
| Funding gap per tCO2e (NPV) | $0.0 |
| Community benefit sharing fund % of revenue | 5% |

### Cost Estimates
| Cost estimates (USD)   | Total cost      | NPV             |
| ---------------------- | --------------- | --------------- |
| **Capital expenditure** | **$2,990,267** | **$2,732,701** |
| Feasibility analysis | $100,000 | $100,000 |
| Conservation planning and admin | $667,067 | $629,559 |
| Data collection and field costs | $80,000 | $76,963 |
| Community representation / liaison | $379,500 | $351,049 |
| Blue carbon project planning | $100,000 | $88,900 |
| Establishing carbon rights | $200,000 | $185,006 |
| Validation | $50,000 | $44,450 |
| Implementation labor | $1,413,700 | $1,256,774 |
| **Operating expenditure** | **$4,729,773** | **$2,916,212** |
| Monitoring | $994,000 | $600,463 |
| Maintenance | $353,284 | $290,522 |
| Community benefit sharing fund | $246,078 | $145,811 |
| Carbon standard fees | $4,412 | $2,665 |
| Baseline reassessment | $120,000 | $75,812 |
| MRV | $400,000 | $223,062 |
| Long-term project operating | $2,612,000 | $1,577,877 |
| **Total cost** | **$7,720,040** | **$5,648,912** |

### Cost Estimates
| Cost estimates (USD)   | Total cost      | NPV             |
| ---------------------- | --------------- | --------------- |
| **Capital expenditure** | **$2,990,267** | **$2,732,701** |
| Feasibility analysis | $100,000 | $100,000 |
| Conservation planning and admin | $667,067 | $629,559 |
| Data collection and field costs | $80,000 | $76,963 |
| Community representation / liaison | $379,500 | $351,049 |
| Blue carbon project planning | $100,000 | $88,900 |
| Establishing carbon rights | $200,000 | $185,006 |
| Validation | $50,000 | $44,450 |
| Implementation labor | $1,413,700 | $1,256,774 |
| **Operating expenditure** | **$4,729,773** | **$2,916,212** |
| Monitoring | $994,000 | $600,463 |
| Maintenance | $353,284 | $290,522 |
| Community benefit sharing fund | $246,078 | $145,811 |
| Carbon standard fees | $4,412 | $2,665 |
| Baseline reassessment | $120,000 | $75,812 |
| MRV | $400,000 | $223,062 |
| Long-term project operating | $2,612,000 | $1,577,877 |
| **Total cost** | **$7,720,040** | **$5,648,912** |

### Project Summary
| Parameter                            | Value                                             |
| ------------------------------------ | ------------------------------------------------- |
| Project | United States Mangrove Restoration (100 ha) |
| $/tCO2e (total cost, NPV) | $256 |
| $/ha | $56,489 |
| NPV covering cost | $-0 |
| IRR when priced to cover opex | 4.0% |
| IRR when priced to cover total costs | -11.7% |
| Total cost (NPV) | $5,648,912 |
| Capital expenditure (NPV) | $2,732,701 |
| Operating expenditure (NPV) | $2,916,212 |
| Credits issued | 22,061 |
| Total revenue (NPV) | $2,916,212 |
| Total revenue (non-discounted) | $4,921,553 |
| Financing cost | $149,513 |
| Funding gap (NPV) | $0 |
| Funding gap per tCO2e (NPV) | $0.0 |
| Community benefit sharing fund % of revenue | 5% |

...

### Cost Estimates
| Cost estimates (USD)   | Total cost      | NPV             |
| ---------------------- | --------------- | --------------- |
| **Capital expenditure** | **$1,430,000** | **$1,342,839** |
| Feasibility analysis | $50,000 | $50,000 |
| Conservation planning and admin | $667,067 | $629,559 |
| Data collection and field costs | $80,000 | $76,963 |
| Community representation / liaison | $332,933 | $314,213 |
| Blue carbon project planning | $100,000 | $88,900 |
| Establishing carbon rights | $150,000 | $138,755 |
| Validation | $50,000 | $44,450 |
| Implementation labor | $0 | $0 |
| **Operating expenditure** | **$4,002,187** | **$2,237,586** |
| Monitoring | $388,000 | $234,386 |
| Maintenance | $0 | $0 |
| Community benefit sharing fund | $2,126,827 | $1,118,793 |
| Carbon standard fees | $49,360 | $26,337 |
| Baseline reassessment | $120,000 | $75,812 |
| MRV | $300,000 | $167,296 |
| Long-term project operating | $1,018,000 | $614,961 |
| **Total cost** | **$5,432,187** | **$3,580,425** |

### Project Summary
| Parameter                            | Value                                             |
| ------------------------------------ | ------------------------------------------------- |
| Project | China Salt marsh Conservation (8000 ha) |
| $/tCO2e (total cost, NPV) | $15 |
| $/ha | $448 |
| NPV covering cost | $0 |
| IRR when priced to cover opex | 4.0% |
| IRR when priced to cover total costs | -6.1% |
| Total cost (NPV) | $3,580,425 |
| Capital expenditure (NPV) | $1,342,839 |
| Operating expenditure (NPV) | $2,237,586 |
| Credits issued | 246,801 |
| Total revenue (NPV) | $2,237,586 |
| Total revenue (non-discounted) | $4,253,654 |
| Financing cost | $71,500 |
| Funding gap (NPV) | $0 |
| Funding gap per tCO2e (NPV) | $0.0 |
| Community benefit sharing fund % of revenue | 50% |

Calculating project China Salt marsh Conservation 8000...
No default Tier 2 sequestration rate data available for Salt marsh.


### Cost Estimates
| Cost estimates (USD)   | Total cost      | NPV             |
| ---------------------- | --------------- | --------------- |
| **Capital expenditure** | **$1,430,000** | **$1,342,839** |
| Feasibility analysis | $50,000 | $50,000 |
| Conservation planning and admin | $667,067 | $629,559 |
| Data collection and field costs | $80,000 | $76,963 |
| Community representation / liaison | $332,933 | $314,213 |
| Blue carbon project planning | $100,000 | $88,900 |
| Establishing carbon rights | $150,000 | $138,755 |
| Validation | $50,000 | $44,450 |
| Implementation labor | $0 | $0 |
| **Operating expenditure** | **$6,412,918** | **$3,505,723** |
| Monitoring | $388,000 | $234,386 |
| Maintenance | $0 | $0 |
| Community benefit sharing fund | $4,537,558 | $2,386,930 |
| Carbon standard fees | $49,360 | $26,337 |
| Baseline reassessment | $120,000 | $75,812 |
| MRV | $300,000 | $167,296 |
| Long-term project operating | $1,018,000 | $614,961 |
| **Total cost** | **$7,842,918** | **$4,848,562** |

### Project Summary
| Parameter                            | Value                                             |
| ------------------------------------ | ------------------------------------------------- |
| Project | China Salt marsh Conservation (8000 ha) |
| $/tCO2e (total cost, NPV) | $20 |
| $/ha | $606 |
| NPV covering cost | $1,268,137 |
| IRR when priced to cover opex | 27.6% |
| IRR when priced to cover total costs | 3.7% |
| Total cost (NPV) | $4,848,562 |
| Capital expenditure (NPV) | $1,342,839 |
| Operating expenditure (NPV) | $3,505,723 |
| Credits issued | 246,801 |
| Total revenue (NPV) | $4,773,860 |
| Total revenue (non-discounted) | $9,075,116 |
| Financing cost | $71,500 |
| Funding gap (NPV) | $0 |
| Funding gap per tCO2e (NPV) | $0.0 |
| Community benefit sharing fund % of revenue | 50% |

In [14]:
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type,country_size_ha,country_code,base_size,...,initial_price_assumption,leftover_after_opex,leftover_after_opex_npv,total_revenue,total_revenu_npv,credits_issued,monitoring_npv,maintenance_npv,monitoring,maintenance
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500,...,190.103739,-4729773.0,-2916212.0,0,0,0,600463,290522,994000,353284
6,United States,Mangrove,Restoration,Hydrology,Small,100,Opex breakeven,9.470846e+08,USA,500,...,260.495904,-7307008.0,-4995044.0,0,0,0,600463,2264746,994000,2753998
12,United States,Mangrove,Restoration,Hybrid,Small,100,Opex breakeven,9.470846e+08,USA,500,...,251.892764,-7102514.0,-4830077.0,0,0,0,600463,2108028,994000,2563424
18,United States,Mangrove,Restoration,Planting,Small,100,Market price,9.470846e+08,USA,500,...,30,-4522529.0,-2793411.0,0,0,0,600463,290522,994000,353284
24,United States,Mangrove,Restoration,Hydrology,Small,100,Market price,9.470846e+08,USA,500,...,30,-6934055.0,-4774054.0,0,0,0,600463,2264746,994000,2753998


In [15]:
# export to csv
df_combinations.to_csv(output_path, index=False)

In [17]:
# export as the projects excel tab
with pd.ExcelWriter(
    master_data_path,
    mode="a",
    engine="openpyxl",
    if_sheet_exists="replace",
) as writer:
    df_combinations.to_excel(
        writer,
        index=False,
        sheet_name="Projects",
    )


## Scorecard rating:

In [ ]:
# Open csv file
df_combinations = pd.read_csv(output_path)
# fill the nan values with 0
df_combinations.fillna(0, inplace=True)
df_combinations.head()

,country,ecosystem,activity,activity_type,project_size_filter,project_size,price_type,country_size_ha,country_code,base_size,...,leftover_after_opex,leftover_after_opex_npv,total_revenue,total_revenu_npv,credits_issued,monitoring_npv,maintenance_npv,monitoring,maintenance,abatement_potential
0,United States,Mangrove,Restoration,Planting,Small,100,Opex breakeven,9.470846e+08,USA,500,...,0.0,0.0,0.0,0.0,0.0,600463.0,290522.0,994000.0,353284.0,166589.0
1,United States,Mangrove,Restoration,Hydrology,Small,100,Opex breakeven,9.470846e+08,USA,500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,United States,Mangrove,Restoration,Hybrid,Small,100,Opex breakeven,9.470846e+08,USA,500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,United States,Mangrove,Restoration,Planting,Small,100,Market price,9.470846e+08,USA,500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,United States,Mangrove,Restoration,Hydrology,Small,100,Market price,9.470846e+08,USA,500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Open scorecard rating uploaded file.
df_scorecard = pd.read_csv(raw_scorecard_data_path)
# fill the strings ' ' with 0
df_scorecard = df_scorecard.replace(" ", 0)
# fill nans with 0
df_scorecard = df_scorecard.fillna(0)
df_scorecard.head()

In [ ]:
# check the dtypes of the columns
df_scorecard.dtypes
# Set the social_feasibility and the coastal_protection_benefit to float
df_scorecard["social_feasibility"] = df_scorecard["social_feasibility"].astype(float)
df_scorecard["coastal_protection_benefit"] = df_scorecard["coastal_protection_benefit"].astype(
    float
)
df_scorecard.head()

In [ ]:
# Add scoredcard rating to the df_combinations

# Columns to add in df_combinations
columns_to_add = [
    "financial_feasibility",
    "legal_feasibility",
    "implementation_risk_score",
    "social_feasibility",
    "availability_of_experienced_labor",
    "security_rating",
    "availability_of_alternative_funding",
    "coastal_protection_benefit",
    "biodiversity_benefit",
]
df_combinations[columns_to_add] = None

for index, row in df_combinations.iterrows():
    country = row["country"]
    ecosystem = row["ecosystem"]
    # 1. Category: Economic:
    # 1.1 Financial_feasibility (weight 20)
    if (
        round(float(row["total_revenue_npv"]) - float(row["opex_npv"]))
        < float(row["opex_npv"]) * -0.25
    ):
        financial_feasibility = 1
    elif round(float(row["total_revenue_npv"]) - float(row["opex_npv"])) < 0:
        financial_feasibility = 2
    else:
        financial_feasibility = 3
    df_combinations.loc[index, "financial_feasibility"] = financial_feasibility
    # Add financial_feasibility_color
    df_combinations.loc[index, "financial_feasibility_color"] = (
        "Low"
        if financial_feasibility == 1
        else "Medium"
        if financial_feasibility == 2
        else "High"
        if financial_feasibility == 3
        else "Not Available"
    )

    # 2. Non-economic
    # 2.1 Legal feasibility (weight 12)
    # Get the legal feasibility score from the scorecard for the country and ecosystem
    legal_feasibility = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["legal_feasibility"].values[0]
    df_combinations.loc[index, "legal_feasibility"] = legal_feasibility
    # Add legal_feasibility_color
    df_combinations.loc[index, "legal_feasibility_color"] = (
        "High"
        if legal_feasibility == 1
        else "Medium"
        if legal_feasibility == 2
        else "Low"
        if legal_feasibility == 3
        else "Not Available"
    )

    # 2.2 Implementation risk score (weight 12)
    # Get the implementation risk score from the scorecard for the country and ecosystem
    implementation_risk_score = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["implementation_risk_score"].values[0]
    df_combinations.loc[index, "implementation_risk_score"] = implementation_risk_score
    # Add implementation_risk_score_color
    df_combinations.loc[index, "implementation_risk_score_color"] = (
        "High"
        if implementation_risk_score == 1
        else "Medium"
        if implementation_risk_score == 2
        else "Low"
        if implementation_risk_score == 3
        else "Not Available"
    )

    # 2.3 Social feasibility (weight 12)
    # Get the social feasibility score from the scorecard for the country and ecosystem
    social_feasibility = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["social_feasibility"].values[0]
    df_combinations.loc[index, "social_feasibility"] = social_feasibility
    # Add social_feasibility_color
    df_combinations.loc[index, "social_feasibility_color"] = (
        "High"
        if social_feasibility == 1
        else "Medium"
        if social_feasibility == 2
        else "Low"
        if social_feasibility == 3
        else "Not Available"
    )

    # 2.4 Availability of experienced labor (weight 10)
    # Get the availability of experienced labor score from the scorecard
    # for the country and ecosystem
    availability_of_experienced_labor = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["availability_of_experienced_labor"].values[0]
    df_combinations.loc[index, "availability_of_experienced_labor"] = (
        availability_of_experienced_labor
    )
    # Add availability_of_experienced_labor_color
    df_combinations.loc[index, "availability_of_experienced_labor_color"] = (
        "High"
        if availability_of_experienced_labor == 1
        else "Medium"
        if availability_of_experienced_labor == 2
        else "Low"
        if availability_of_experienced_labor == 3
        else "Not Available"
    )

    # 2.5 Security rating (weight 5)
    # Get the security rating score from the scorecard for the country and ecosystem
    security_rating = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["security_rating"].values[0]
    df_combinations.loc[index, "security_rating"] = security_rating
    # Add security_rating_color
    df_combinations.loc[index, "security_rating_color"] = (
        "High"
        if security_rating == 1
        else "Medium"
        if security_rating == 2
        else "Low"
        if security_rating == 3
        else "Not Available"
    )

    # 2.6 Availability of alternative funding (weight 5)
    # Get the availability of alternative funding score from the scorecard
    # for the country and ecosystem
    availability_of_alternative_funding = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["availability_of_alternative_funding"].values[0]
    df_combinations.loc[index, "availability_of_alternative_funding"] = (
        availability_of_alternative_funding
    )
    # Add availability_of_alternative_funding_color
    df_combinations.loc[index, "availability_of_alternative_funding_color"] = (
        "High"
        if availability_of_alternative_funding == 1
        else "Medium"
        if availability_of_alternative_funding == 2
        else "Low"
        if availability_of_alternative_funding == 3
        else "Not Available"
    )

    # 2.7 Coastal protection benefit (weight 3)
    # Get the coastal protection benefit score from the scorecard for the country and ecosystem
    coastal_protection_benefit = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["coastal_protection_benefit"].values[0]
    df_combinations.loc[index, "coastal_protection_benefit"] = coastal_protection_benefit
    # Add coastal_protection_benefit_color
    df_combinations.loc[index, "coastal_protection_benefit_color"] = (
        "High"
        if coastal_protection_benefit == 1
        else "Medium"
        if coastal_protection_benefit == 2
        else "Low"
        if coastal_protection_benefit == 3
        else "Not Available"
    )

    # 2.8 Biodiversity benefit (weight 3)
    # Get the biodiversity benefit score from the scorecard for the country and ecosystem
    biodiversity_benefit = df_scorecard[
        (df_scorecard["country"] == country) & (df_scorecard["ecosystem"] == ecosystem)
    ]["biodiversity_benefit"].values[0]
    df_combinations.loc[index, "biodiversity_benefit"] = biodiversity_benefit
    # Add biodiversity_benefit_color
    df_combinations.loc[index, "biodiversity_benefit_color"] = (
        "High"
        if biodiversity_benefit == 1
        else "Medium"
        if biodiversity_benefit == 2
        else "Low"
        if biodiversity_benefit == 3
        else "Not Available"
    )

    # if value is Nan, set to 0 for the calculation
    if math.isnan(legal_feasibility):
        legal_feasibility = 0
    if math.isnan(implementation_risk_score):
        implementation_risk_score = 0
    if math.isnan(social_feasibility):
        social_feasibility = 0
    if math.isnan(availability_of_experienced_labor):
        availability_of_experienced_labor = 0
    if math.isnan(security_rating):
        security_rating = 0
    if math.isnan(availability_of_alternative_funding):
        availability_of_alternative_funding = 0
    if math.isnan(coastal_protection_benefit):
        coastal_protection_benefit = 0
    if math.isnan(biodiversity_benefit):
        biodiversity_benefit = 0

    # Add scorecard_rating
    scorecard_rating = (
        (float(legal_feasibility) * 0.12)
        + (float(implementation_risk_score) * 0.12)
        + (float(social_feasibility) * 0.12)
        + (float(security_rating) * 0.05)
        + (float(availability_of_experienced_labor) * 0.10)
        + (float(availability_of_alternative_funding) * 0.05)
        + (float(coastal_protection_benefit) * 0.03)
        + (float(biodiversity_benefit) * 0.03)
    ) / (0.12 + 0.12 + 0.12 + 0.05 + 0.10 + 0.05 + 0.03 + 0.03)
    df_combinations.loc[index, "scorecard_rating"] = scorecard_rating
    if scorecard_rating > 1 and scorecard_rating <= 1.666:
        df_combinations.loc[index, "scorecard_rating_color"] = "High"
    elif scorecard_rating > 1.666 and scorecard_rating <= 2.333:
        df_combinations.loc[index, "scorecard_rating_color"] = "Medium"
    elif scorecard_rating > 2.333 and scorecard_rating <= 3:
        df_combinations.loc[index, "scorecard_rating_color"] = "Low"
    else:
        df_combinations.loc[index, "scorecard_rating_color"] = "Not Available"

df_combinations.head()

In [ ]:
# export to csv
df_combinations.to_csv(output_scorecard, index=False)